In [3]:
import sqlite3

# conn = sqlite3.connect("/home/alex/network_workdir/elections/databases_ssd/complete_trump_vs_hillary_db.sqlite")
conn = sqlite3.connect("/home/alex/network_workdir/elections/databases_ssd/complete_trump_vs_hillary_sep-nov_db.sqlite")
c = conn.cursor()
# c.execute("SELECT count(*) FROM tweet_to_retweeted_uid LIMIT 1;")

In [4]:
c.execute('''SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;''')
for i in c.fetchall():
    print(i)

('class_proba',)
('filename',)
('hashtag',)
('hashtag_tweet_user',)
('query',)
('retweeted_status',)
('source_content',)
('source_url',)
('tweet',)
('tweet_to_keyword',)
('tweet_to_mentioned_uid',)
('tweet_to_query_id',)
('tweet_to_quoted_uid',)
('tweet_to_replied_uid',)
('tweet_to_retweeted_uid',)
('url_retweet_user',)
('url_tweet_user',)
('user',)


In [6]:
def describe(table):
    print("-" * 50, "TABLE", table, "-" * 50)
    c.execute("SELECT count(*) FROM '{}';".format(table))
    d = c.fetchone()
    print('count：', d)

    c.execute("SELECT * FROM '{}' LIMIT 1;".format(table))
    col_name = [t[0] for t in c.description]
    d = c.fetchone()
    for k, v in zip(col_name, d):
        print(k, v, sep="\t\t")
    print("")
        
c.execute('''SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;''')
for table in c.fetchall():
    describe(table[0])

-------------------------------------------------- TABLE class_proba --------------------------------------------------
count： (97758182,)
tweet_id		771131463345565696
user_id		2429107224
p_pro_hillary_anti_trump		0.010749551864870628

-------------------------------------------------- TABLE filename --------------------------------------------------
count： (1353,)
id		0
filename		tweets-c1dcab45-e68e-4861-a897-5c9c8ecd5f19.taj

-------------------------------------------------- TABLE hashtag --------------------------------------------------
count： (571629,)
id		1
hashtag		werk
count		25

-------------------------------------------------- TABLE hashtag_tweet_user --------------------------------------------------
count： (43655958,)
tweet_id		771131463383285761
hashtag		usa
user_id		3192506298
ht_signi_final		None
ht_signi_final_june_sep		None

-------------------------------------------------- TABLE query --------------------------------------------------
count： (2,)
id		0
query		hill

In [1]:
# 关键fake news

tid = 742417158429233152

c.execute('''SELECT * FROM tweet WHERE tweet_id={};'''.format(tid))
col_name = [t[0] for t in c.description]
d = c.fetchone()
print(d)
for k, v in zip(col_name, d):
    print(k, v, sep="\t")
print("-" * 100)

c.execute('''SELECT * FROM retweeted_status WHERE tweet_id={};'''.format(tid))
col_name = [t[0] for t in c.description]
d = c.fetchone()
for k, v in zip(col_name, d):
    print(k, v, sep="\t")
print("-" * 100)

c.execute('''SELECT * FROM class_proba WHERE tweet_id={};'''.format(tid))
col_name = [t[0] for t in c.description]
d = c.fetchone()
if d:
    for k, v in zip(col_name, d):
        print(k, v, sep="\t")
else:
    print("是原创的tweet！")
print("-" * 100)

c.execute('''SELECT * FROM url_tweet_user WHERE tweet_id={};'''.format(tid))
col_name = [t[0] for t in c.description]
d = c.fetchone()
for k, v in zip(col_name, d):
    print(k, v, sep="\t")

c.execute('''SELECT * FROM user WHERE user_id={};'''.format(39344374))
col_name = [t[0] for t in c.description]
d = c.fetchone()
for k, v in zip(col_name, d):
    print(k, v, sep="\t")
print("-" * 100)

NameError: name 'c' is not defined

In [2]:
conn = sqlite3.connect("/home/alex/network_workdir/elections/databases/urls_db.sqlite")
c = conn.cursor()

### c.execute('''SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;''')
for d in c.fetchall():
    print(d)
    
c.execute('''SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;''')
for table in c.fetchall():
    describe(table[0])

NameError: name 'sqlite3' is not defined

In [16]:
import ujson as json

# 发现那些其实是转发的tweet！
source_tids = []
data = json.load(open("data/retweet_network_fake.json"))
for source in data.values():
    c.execute("select retweeted_status from urls where tweet_id={}".format(source))
    d = c.fetchone()
    if d and d[0] == 0:
        source_tids.append(source)
    
print(len(source_tids))

386802


In [17]:
print(source_tids[0])

set_tids = set(source_tids)

cnt = 0
for source in data.keys():
    if source in set_tids:
        cnt += 1
print(cnt)

766707481611497472
0


In [13]:
from tqdm import tqdm

conn = sqlite3.connect("/home/alex/network_workdir/elections/databases_ssd/complete_trump_vs_hillary_db.sqlite")
c = conn.cursor()

for _id in tqdm(source_tids):
    c.execute('''SELECT retweet_id FROM tweet_to_retweeted_uid WHERE tweet_id={};'''.format(_id))
    d = c.fetchone()
    if d:
        print(d)

100%|██████████| 1276639/1276639 [00:13<00:00, 96158.37it/s] 
